### Modelo NLP para predecir ODS's

##### Usaremos varios modelos para ver cuál es el más efectivo

In [1]:
import pandas as pd
import numpy as np
import re
import string
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLkNN
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from langdetect import detect, DetectorFactory
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import LabelPowerset
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as skm
#import es_core_web_sm

In [2]:
# Vamos a importar nuestro dataset de datos

contenidos = pd.read_excel(r"C:\Users\pueygu01\Python Programs\Máster - Proyecto\dataset - español y catalan final version.xlsx")
contenidos.head()

español  \
0  erradicar la pobreza extrema para todas las pe...   
1  reducir al menos a la mitad la proporción de h...   
2  garantizar que todos los hombres y mujeres, en...   
3  omentar la resiliencia de los pobres y las per...   
4  reducir su exposición y vulnerabilidad a los f...   

                                             catalan Etiqueta  
0  erradicar la pobresa extrema per a totes les p...     ODS1  
1  reduir almenys a la meitat la proporció d'home...     ODS1  
2  garantir que tots els homes i dones, en partic...     ODS1  
3  omentar la resiliència dels pobres i les perso...     ODS1  
4  reduir la seva exposició i vulnerabilitat als ...     ODS1

In [3]:
# Podemos limpiar un poco el texto de mayúsculas y comillas. Para ello usamos una función que nos lo haga en todo el dataset

def clean_text(text):
    text = text.lower()
    text = text.strip(' ')
    text = text.strip('"')
    return text

In [4]:
# Aplicamos la función anterior
contenidos["español_limpio"]=contenidos.español.apply(clean_text)
contenidos["catalan_limpio"]=contenidos.catalan.apply(clean_text)
contenidos.head()

español  \
0  erradicar la pobreza extrema para todas las pe...   
1  reducir al menos a la mitad la proporción de h...   
2  garantizar que todos los hombres y mujeres, en...   
3  omentar la resiliencia de los pobres y las per...   
4  reducir su exposición y vulnerabilidad a los f...   

                                             catalan Etiqueta  \
0  erradicar la pobresa extrema per a totes les p...     ODS1   
1  reduir almenys a la meitat la proporció d'home...     ODS1   
2  garantir que tots els homes i dones, en partic...     ODS1   
3  omentar la resiliència dels pobres i les perso...     ODS1   
4  reduir la seva exposició i vulnerabilitat als ...     ODS1   

                                      español_limpio  \
0  erradicar la pobreza extrema para todas las pe...   
1  reducir al menos a la mitad la proporción de h...   
2  garantizar que todos los hombres y mujeres, en...   
3  omentar la resiliencia de los pobres y las per...   
4  reducir su exposición y vulnerabilidad a los f...   

                                      catalan_limpio  
0  erradicar la pobresa extrema per a totes les p...  
1  reduir almenys a la meitat la proporció d'home...  
2  garantir que tots els homes i dones, en partic...  
3  omentar la resiliència dels pobres i les perso...  
4  reduir la seva exposició i vulnerabilitat als ...

In [5]:
# Para poder aplicar un modelo de prediccón de multi-etiqueta, es necesario que las etiquetas sean un conjunto de listas
# Pasamos a lista nuestras etiquetas

contenidos["Etiqueta"] = contenidos["Etiqueta"].str.split()

In [6]:
# Con este paquete podemos extraer las etiquetas y ponerlas como columnas ya que es así cómo los modelos pueden trabajr con el dataset

multilabel = MultiLabelBinarizer()
new_contenidos = multilabel.fit_transform(contenidos["Etiqueta"])


In [7]:
# Para ver todas las diferentes etiquetas, llamamos a "classes_" mediante miltilabel
multilabel.classes_

array(['ODS1', 'ODS10', 'ODS11', 'ODS12', 'ODS13', 'ODS14', 'ODS15',
       'ODS16', 'ODS17', 'ODS2', 'ODS3', 'ODS4', 'ODS5', 'ODS6', 'ODS7',
       'ODS8', 'ODS9'], dtype=object)

In [8]:
# Para poder trabajar, lo tenemos que pasar a DataFrame y aprovechamos para poner las etiquetas como columnas

new_contenidos1 = pd.DataFrame(new_contenidos
             , columns = multilabel.classes_)
new_contenidos1.head()

ODS1  ODS10  ODS11  ODS12  ODS13  ODS14  ODS15  ODS16  ODS17  ODS2  ODS3  \
0     1      0      0      0      0      0      0      0      0     0     0   
1     1      0      0      0      0      0      0      0      0     0     0   
2     1      0      0      0      0      0      0      0      0     0     0   
3     1      0      0      0      0      0      0      0      0     0     0   
4     1      0      0      0      0      0      0      0      0     0     0   

   ODS4  ODS5  ODS6  ODS7  ODS8  ODS9  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     0     0     0     0  
3     0     0     0     0     0     0  
4     0     0     0     0     0     0

In [9]:
# Concatenamos el último dataframe con su correspondiente comentario para obtener así el formato de tabla que necesitamos
# para moderlar

df_new_contenidos = pd.concat([contenidos, new_contenidos1], axis=1)

df_new_contenidos.head()

español  \
0  erradicar la pobreza extrema para todas las pe...   
1  reducir al menos a la mitad la proporción de h...   
2  garantizar que todos los hombres y mujeres, en...   
3  omentar la resiliencia de los pobres y las per...   
4  reducir su exposición y vulnerabilidad a los f...   

                                             catalan Etiqueta  \
0  erradicar la pobresa extrema per a totes les p...   [ODS1]   
1  reduir almenys a la meitat la proporció d'home...   [ODS1]   
2  garantir que tots els homes i dones, en partic...   [ODS1]   
3  omentar la resiliència dels pobres i les perso...   [ODS1]   
4  reduir la seva exposició i vulnerabilitat als ...   [ODS1]   

                                      español_limpio  \
0  erradicar la pobreza extrema para todas las pe...   
1  reducir al menos a la mitad la proporción de h...   
2  garantizar que todos los hombres y mujeres, en...   
3  omentar la resiliencia de los pobres y las per...   
4  reducir su exposición y vulnerabilidad a los f...   

                                      catalan_limpio  ODS1  ODS10  ODS11  \
0  erradicar la pobresa extrema per a totes les p...     1      0      0   
1  reduir almenys a la meitat la proporció d'home...     1      0      0   
2  garantir que tots els homes i dones, en partic...     1      0      0   
3  omentar la resiliència dels pobres i les perso...     1      0      0   
4  reduir la seva exposició i vulnerabilitat als ...     1      0      0   

   ODS12  ODS13  ...  ODS16  ODS17  ODS2  ODS3  ODS4  ODS5  ODS6  ODS7  ODS8  \
0      0      0  ...      0      0     0     0     0     0     0     0     0   
1      0      0  ...      0      0     0     0     0     0     0     0     0   
2      0      0  ...      0      0     0     0     0     0     0     0     0   
3      0      0  ...      0      0     0     0     0     0     0     0     0   
4      0      0  ...      0      0     0     0     0     0     0     0     0   

   ODS9  
0     0  
1     0  
2     0  
3     0  
4     0  

[5 rows x 22 columns]

# Creación de Modelos
------------------------

### `Definición de molelos en Castellano`

In [10]:
print(df_new_contenidos.columns[5:])

Index(['ODS1', 'ODS10', 'ODS11', 'ODS12', 'ODS13', 'ODS14', 'ODS15', 'ODS16',
       'ODS17', 'ODS2', 'ODS3', 'ODS4', 'ODS5', 'ODS6', 'ODS7', 'ODS8',
       'ODS9'],
      dtype='object')


In [11]:
# Definimos un vector X con los comentarios y un vector y con las etiquetas (1 ó 0)

X_castellano = df_new_contenidos["español_limpio"]
y_castellano = np.asarray(df_new_contenidos[df_new_contenidos.columns[5:]])

# Iniciamos el paquete TfidfVectorizer 

vetorizar_castellano = TfidfVectorizer(max_features=3000, max_df=0.85)

# Aplicamos el fit() en el vector de comentarios X
vetorizar_castellano.fit(X_castellano)

# Usando "train_test_split" partimos nuestro dataset en entrenamiento y test
X_train_castellano, X_test_castellano, y_train_castellano, y_test_castellano = train_test_split(X_castellano
                                                                                               ,y_castellano
                                                                                               ,test_size=0.25
                                                                                               ,random_state=42)

# Transformamos nuestros datos

X_castellano_train_tfidf = vetorizar_castellano.transform(X_train_castellano)
X_castellano_test_tfidf = vetorizar_castellano.transform(X_test_castellano)

#################################################################################
############## Clasificación Multilabel de los vecinos más cercanos #############
#################################################################################

mlknn_classifier_castellano = MLkNN()
mlknn_classifier_castellano.fit(X_castellano_train_tfidf, y_train_castellano)

#################################################################################
######################## Relevancia Binaria con Naive Bayes #####################
#################################################################################

bin_class_castellano = BinaryRelevance(GaussianNB())
bin_class_castellano.fit(X_castellano_train_tfidf, y_train_castellano)

#################################################################################
##################### Power Label Set con Regresión logística ###################
#################################################################################

label_powerset_castellano = LabelPowerset(LogisticRegression())
label_powerset_castellano.fit(X_castellano_train_tfidf, y_train_castellano)


C:\Users\pueygu01\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_neighbors=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


LabelPowerset(classifier=LogisticRegression(), require_dense=[True, True])

### `Definición de molelos en Catalán`

In [12]:
# Definimos un vector X con los comentarios y un vector y con las etiquetas (1 ó 0)

X_catalan = df_new_contenidos["catalan_limpio"]
y_catalan = np.asarray(df_new_contenidos[df_new_contenidos.columns[5:]])

# Iniciamos el paquete TfidfVectorizer 

vetorizar_catalan = TfidfVectorizer(max_features=3000, max_df=0.85)

# Aplicamos el fit() en el vector de comentarios X
vetorizar_catalan.fit(X_catalan)

# Usando "train_test_split" partimos nuestro dataset en entrenamiento y test
X_train_catalan, X_test_catalan, y_train_catalan, y_test_catalan = train_test_split(X_catalan
                                                                                   ,y_catalan
                                                                                   ,test_size=0.25
                                                                                   ,random_state=42)

# Transformamos nuestros datos

X_catalan_train_tfidf = vetorizar_catalan.transform(X_train_catalan)
X_catalan_test_tfidf = vetorizar_catalan.transform(X_test_catalan)

#################################################################################
############## Clasificación Multilabel de los vecinos más cercanos #############
#################################################################################

mlknn_classifier_catalan = MLkNN()
mlknn_classifier_catalan.fit(X_catalan_train_tfidf, y_train_catalan)

#################################################################################
######################## Relevancia Binaria con Naive Bayes #####################
#################################################################################

bin_class_catalan = BinaryRelevance(GaussianNB())
bin_class_catalan.fit(X_catalan_train_tfidf, y_train_catalan)

#################################################################################
##################### Power Label Set con Regresión logística ###################
#################################################################################

label_powerset_catalan = LabelPowerset(LogisticRegression())
label_powerset_catalan.fit(X_catalan_train_tfidf, y_train_catalan)



C:\Users\pueygu01\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_neighbors=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


LabelPowerset(classifier=LogisticRegression(), require_dense=[True, True])

### Generación de las dos funciones. Una para cada idioma
Las siguientes funciones se usarán para llamar al modelo entrenado y predecir ODSs en castellano o catalán

In [13]:
def identificador_ODS_castellano(comentario):
    frase_tfidf = vetorizar_castellano.transform(comentario)
    predicción = multilabel.inverse_transform(mlknn_classifier_castellano.predict(frase_tfidf))
    return print("El/los ODS para ese comentario son;", predicción)

def identificador_ODS_catalan(comentario):
    frase_tfidf = vetorizar_catalan.transform(comentario)
    predicción = multilabel.inverse_transform(mlknn_classifier_catalan.predict(frase_tfidf))
    return print("El/los ODS para ese comentario son;", predicción)

## Cargamos el dataset de la aplicació

In [14]:
txt = pd.read_excel(r"C:\Users\pueygu01\Python Programs\Máster - Proyecto\contenidos y etiquetas_proba.xlsx", index_col=None)
txt.head(10)

title  \
0                      La SEXUALITAT AL TEU ABAST ??   
1  Redescobreix el nostre entorn natural amb "La ...   
2                  MULLA'T PER L'ESCLEROSI MÚLTIPLE.   
3            3a Gala Lírica "Els Claustres de Breda"   

                                         description Etiqueta  
0  Taller de Sexualitat: “El plaer masculí” amb l...     ODS5  
1  ? Vols reconnectar amb la natura? Estimula tot...    ODS13  
2  Bon dia a tothom, com cada any per aquestes da...     ODS3  
3  Llancem una iniciativa per ajudar al menjador ...      NaN

# Función para limpiar el texto

In [15]:
#lista de stop-words específicos de nuestro corpus (aproximación)
stop_words = ['el', 'la', 'lo', 'los', 'las', 'un', 'una', 'unos', 'unas', 'me', 'a', 'de', 'se', 'te','?']

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    eliminamos los tokens que son signos de puntuación, convertimos en
    minúsculas y quitamos signos de puntuación. Para terminar
    volvemos a convertir en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lower_ for tok in tokens if not tok.is_punct and not tok.is_space]
    filtered_tokens = [pattern2.sub('', token) for token in tokens if not (token in stop_words)]
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text

def lemmatize_text(text):
    """Convertimos el texto a tokens, extraemos el lema de cada token
    y volvemos a convertir en cadena de texto"""
    tokens = nlp(text)
    lemmatized_tokens = [tok.lemma_ for tok in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    
    return lemmatized_text

## Iterable por el dat set de la aplicación y cuando encuentra, en la columna del ODS que esta vacio entonces aplcamos el modelo predictivo

In [16]:
DetectorFactory.seed = 0
for t in txt.index:
    if pd.isnull(txt['Etiqueta'][t]):
        lengua=detect(txt['description'][t])
        if lengua=='ca':
            nlp = spacy.load("ca_core_news_sm")
            txt["lemas"]=txt.description.apply(clean_text)
            txt["lemas"]=txt.lemas.apply(lemmatize_text)
            identificador_ODS_catalan([txt['description'][t]])
        else:
            nlp = spacy.load("es_core_news_sm")
            txt.description.apply(clean_text)
            txt.description.apply(lemmatize_text)
            identificador_ODS_castellano([txt['description'][t]])
    

print(lengua)

El/los ODS para ese comentario son; [('ODS1', 'ODS2')]
ca


# Rendimiento de lo modelos


-------------------------------------

### MLkNN - Vecinos más cercanos

#### `Castellano`

In [17]:
prediccion_castellano = mlknn_classifier_castellano.predict(X_castellano_test_tfidf)
print("El accuracy del modelo es ;",accuracy_score(y_test_castellano,prediccion_castellano))

El accuracy del modelo es ; 0.5647382920110193


#### `Catalán`

In [18]:
prediccion_catalan = mlknn_classifier_catalan.predict(X_catalan_test_tfidf)
print("El accuracy del modelo es ;",accuracy_score(y_test_catalan,prediccion_catalan))

El accuracy del modelo es ; 0.5426997245179064


### Relevancia binaria


#### `Castellano`

In [19]:
prediccion_castellano = bin_class_castellano.predict(X_castellano_test_tfidf)
print("El accuracy del modelo es ;",accuracy_score(prediccion_castellano, y_test_castellano))

El accuracy del modelo es ; 0.49586776859504134


#### `Catalan`

In [20]:
prediccion_catalan = bin_class_catalan.predict(X_catalan_test_tfidf)
print("El accuracy del modelo es ;",accuracy_score(prediccion_catalan, y_test_catalan))

El accuracy del modelo es ; 0.4820936639118457


### Label Powerset con regresión logística

#### `Castellano`

In [21]:
prediccion_castellano = label_powerset_castellano.predict(X_castellano_test_tfidf)
print("El accuracy del modelo es ;",accuracy_score(prediccion_castellano, y_test_castellano))

El accuracy del modelo es ; 0.650137741046832


#### `Catalán`

In [22]:
prediccion_catalan = label_powerset_catalan.predict(X_catalan_test_tfidf)
print("El accuracy del modelo es ;",accuracy_score(prediccion_catalan, y_test_catalan))

El accuracy del modelo es ; 0.6473829201101928


In [23]:
prediccion_castellano = bin_class_catalan.predict(X_catalan_test_tfidf)
#cm = skm.multilabel_confusion_matrix(y_test_castellano,prediccion_castellano)
#print(cm)
print(skm.classification_report(y_test_catalan,prediccion_castellano))

              precision    recall  f1-score   support

           0       0.89      0.79      0.84        42
           1       0.76      0.36      0.49        36
           2       0.52      0.48      0.50        23
           3       1.00      0.40      0.57         5
           4       0.56      0.38      0.45        39
           5       0.74      0.40      0.52        42
           6       0.72      0.47      0.57        38
           7       0.76      0.71      0.73        31
           8       0.80      0.33      0.47        12
           9       1.00      0.75      0.86        24
          10       0.96      0.89      0.92        27
          11       0.85      0.74      0.79        31
          12       0.79      0.39      0.52        28
          13       1.00      0.43      0.60         7
          14       0.00      0.00      0.00         1
          15       0.65      0.50      0.57        26
          16       0.85      0.46      0.59        24

   micro avg       0.78   

C:\Users\pueygu01\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pueygu01\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
prediccion_castellano = mlknn_classifier_castellano.predict(X_castellano_test_tfidf)
#conf_matrix = skm.multilabel_confusion_matrix(y_test_castellano,prediccion_castellano)
#print(conf_matrix)
print( skm.classification_report(y_test_castellano,prediccion_castellano))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88        42
           1       0.71      0.33      0.45        36
           2       0.79      0.65      0.71        23
           3       0.00      0.00      0.00         5
           4       0.77      0.51      0.62        39
           5       1.00      0.57      0.73        42
           6       0.83      0.53      0.65        38
           7       0.93      0.81      0.86        31
           8       0.83      0.42      0.56        12
           9       0.85      0.92      0.88        24
          10       0.78      0.93      0.85        27
          11       1.00      0.77      0.87        31
          12       0.81      0.46      0.59        28
          13       0.83      0.71      0.77         7
          14       0.00      0.00      0.00         1
          15       0.70      0.54      0.61        26
          16       0.79      0.46      0.58        24

   micro avg       0.84   

In [25]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from dash.dependencies import Input, Output, State

In [26]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1(children = "Radar ODS", style = {'text-align': 'center',
                                             'color':'blue',
                                             'fontSize': 50}),
    html.Div([
        html.P('Prototipo para predecir Objetivos de Desarrollo Sostenible', style = {'text-align': 'center',
                                                                                      'fontSize': 40}),
        html.P("Lista de Objetivos de Desarrollo Sostenible", style = {'fontSize': 25})
    
    ]),
    
    dcc.Markdown(
        ''' 
            * ODS1 Fin de la pobreza
            * ODS2 Hambre cero
            * ODS3 Salud y bienestar
            * ODS4 Educación de calidad
            * ODS5 Igualdad de género
            * ODS6 Agua y saneamiento
            * ODS7 Energía sostenible y no contaminante
            * ODS8 Trabajo decente y crecimiento económico
            * ODS9 Industria, innovación e infraestructura
            * ODS10 Reducción de las desigualdades
            * ODS11 Ciudades y comunidades sostenibles
            * ODS12 Producción y consumos responsables
            * ODS13 Acción por el clima
            * ODS14 Vida submarina
            * ODS15 Vida de ecosistemas terrestres
            * ODS16 Paz, jusicia e instituciones sólidas
            * ODS17 Alianzas para lograr los objetivos
        '''
    ),
    dcc.Markdown(
        '''*En el cuadro de texto podemos escribir el comentario deseado, ya sea una actividad,
          una iniciativa, una protesta etc*''',style = {'fontSize': 20}),
        
    dcc.Textarea(
        placeholder = 'Introduce la descripción de tu actividad',
        id='textarea-state-example',
        style={'width': '40%', 'height': 100},
    ),
    
    html.Button('Analizar predicción', id='textarea-state-example-button'
                , n_clicks=0
                , style = {'font-size': '20px', 'color':'blue'}),
    
    html.Div(id='textarea-state-example-output'
             , style={'whiteSpace': 'pre-line'})
])

@app.callback(
    Output('textarea-state-example-output', 'children'),
    Input('textarea-state-example-button', 'n_clicks'),
    State('textarea-state-example', 'value')
)
    
def identificador_ODS(n_clicks, value):
    value1 = [value]
    lengua = detect(value)
    if lengua == 'ca':
        frase_tfidf = vetorizar_catalan.transform(value1)
        predicción = multilabel.inverse_transform(mlknn_classifier_catalan.predict(frase_tfidf))
        if n_clicks>0:
            return 'El ODS predicho es: \n{}'.format(predicción)
    else:
        frase_tfidf = vetorizar_castellano.transform(value1)
        predicción = multilabel.inverse_transform(mlknn_classifier_castellano.predict(frase_tfidf))
        if n_clicks>0:
            return 'El ODS predicho es: \n{}'.format(predicción)


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


### Lo vamos a enganchar a la tabla contents para usar el último comentario desde la app

In [27]:
#### Leemos la tabla del servidor usando SQL y cogemos el útlimo comentario

Query para obtener el último registro de la tabla 

SELECT TOP 1000 * FROM table_name ORDER BY column_name DESC


#### Creamos la función para aplicar el modelo si es catlán o castellano

def identificador_ODS(comentario):
    value1 = [comentario]
    lengua = detect(comentario)
    if lengua == 'ca':
        frase_tfidf = vetorizar_catalan.transform(value1)
        predicción = multilabel.inverse_transform(mlknn_classifier_catalan.predict(frase_tfidf))
        return 'El ODS predicho es: \n{}'.format(predicción)
    else:
        frase_tfidf = vetorizar_castellano.transform(value1)
        predicción = multilabel.inverse_transform(mlknn_classifier_castellano.predict(frase_tfidf))
        return 'El ODS predicho es: \n{}'.format(predicción)


SyntaxError: invalid syntax (<ipython-input-27-8b859d91a929>, line 3)

In [31]:
import joblib
joblib.dump(mlknn_classifier_catalan, 'predictor ODS catalan.pkl')
joblib.dump(mlknn_classifier_castellano, 'predictor ODS castellano.pkl')

['predictor ODS castellano.pkl']

In [29]:
import joblib